In [7]:
import re
from pathlib import Path
import shutil
from nipype.interfaces.fsl.maths import ApplyMask
import os

In [8]:
local = True
send = False

data_path = Path('../data')
data_outliers = Path("../data/outliers")
data_claes_shared = Path("/homes/claes/data_shared/marius")

In [9]:
# new outliers

anonymized = [
    'Cmet0132',
    'Cmet0619',
    'Cmet0357',
    'Cmet0146',
    'Cmet0280',
    'Cmet1474'
]

for pt in data_path.glob('nii/anonymized/*'):
    pt_name = str(pt)[-8:]
    if pt_name in anonymized:
        dst = data_path / f'new_outliers'
        print(f'moving {pt_name}\n\tfrom {pt}\n\tto{dst}')
        # if not dst.exists():
        #     dst.mkdir(parents=True, exist_ok=True)
        shutil.move(pt, dst)

In [ ]:
if send:
    if local:
        with open("../notes/data_outliers") as f:
            lines = f.readlines()

        for line in lines:
            match = re.search(r"^(\.\./data/nii/\w+/(Cmet\d+))", line)

            if match:
                pt_folder = match.group(1)
                pt_name = match.group(2)

                src = pt_folder
                dst = Path("../data/outliers") / pt_name

                shutil.move(src, dst)       
    else:
        for pt in Path("../data/outliers").glob("*"):
            src = pt / 'CT.nii.gz'
            dst = Path("/homes/claes/data_shared/marius/data_outliers") / f"{str(pt)[-8:]}.nii.gz"

            shutil.copy(src, dst)
else:
    CT_special = [
        '0209',
        '0255',
        '0283'
        '0289',
        '0296',
        '0441',
        '0519',
        '0937',
        '0721',
        '0801'
    ]

    CT_delete = [
        '0376',
        '0479',
        '1247',
        '1389',
        '1689',
        '0829',
        '1520',
        '1553',
        '1565'
    ]

    for pt in data_outliers.glob('*'):
        pt_nr = str(pt)[-4:]
        print(f"Cmet{pt_nr}")

        if pt_nr in CT_delete:
            shutil.rmtree(pt)
            continue

        # for f in pt.glob('*'):
        #     match = re.search(r"^.*/(CT|PET)\.nii\.gz$", str(f))

        #     if not match:
        #         print(f)
        #         os.remove(f)

        if pt_nr in CT_special:
            print(f"create PET_crop for patient Cmet{pt_nr}")
            continue

        if not (pt / "CT_crop.nii.gz").exists():
            shutil.copy(data_claes_shared / f"formatted/Cmet_{pt_nr}_0000.nii.gz", pt / "CT_crop.nii.gz")

        if not (pt / "ANAT_smoothed_0-100_BET_mask.nii.gz").exists():
            shutil.copy(data_claes_shared / f"CTBET_pp/Cmet{pt_nr}.nii.gz", pt / "ANAT_smoothed_0-100_BET_mask.nii.gz")

        if not (pt / "CT_BET.nii.gz").exists():
            mask = ApplyMask()
            mask.inputs.in_file = f"{pt}/CT_crop.nii.gz"
            mask.inputs.mask_file = f"{pt}/ANAT_smoothed_0-100_BET_mask.nii.gz"
            mask.inputs.out_file = f"{pt}/CT_BET.nii.gz"
            mask.run()
        
        category = "anonymized" if Path(f"../data/rtss/Cmet{pt_nr}").exists() else "controls"
        
        for f in pt.glob('*'):
            to_folder = Path(f"../data/nii/{category}/Cmet{pt_nr}/")
            to_folder.mkdir(parents=True, exist_ok=True)
            shutil.copy(f, to_folder)